In [1]:
import numpy as np
import functools as ft
import networkx as nx

In [2]:
def hdv(d):
    return np.random.choice([-1, 1], d)


def zero(d):
    return np.zeros((d,))


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def p(d):
    return np.random.shuffle(np.eye(d))


def inverse(P):
    return np.linalg.inv(P)


def permute(P, H):
    return P.dot(H)


def cosine_similarity(A, B):
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return np.dot(A, B) / (norm_A * norm_B)


def similarity(A, B):
    return np.dot(A, B) / len(A)


sim = cosine_similarity

In [3]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def getVector(self, label):
        return self.vectors[label]

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: sim(V, x[1]))

In [4]:
def initVertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [5]:
def initNodeMem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.addVector(f"mem{n}", graph.nodes[n]["mem"])

In [6]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if sim(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if sim(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.addVector(f"mem{n}_{i}", mem)

In [7]:
def initGraph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.addVector("g", G)
    return G

In [8]:
def checkEdge(G, A, B, threshold):
    return sim(B, bind([G, A])) > threshold

In [9]:
def nodeMemoryReconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = nodeMemoryReconstruction(G, xs, iter - 1)

    newMems = []
    b = [bind(x) for x in zip(mems, xs)]
    for i in range(len(xs)):
        bmem = bundle(b[:i] + b[i + 1 :])
        newMems.append(bind([xs[i], bundle([G, -bmem])]))

    return newMems

In [10]:
def graphMemoryReconstruction(G, xs, iter=15):
    raise NotImplementedError
    if iter == 0:
        return (G, zeros(len(G)))

    (Gi, Ni) = graphMemoryReconstruction(G, xs, iter - 1)
    Gii = bundle([Gi, -Ni])

    for i in range(iter):
        print(i)

    return bundle([Gi, -edges])

In [93]:
# visited = dict()
# visited = {}


def shortestPath(G, A, B, xs, distance, threshold, memory):
    if np.array_equiv(A, B):
        return 0

    label, value = memory.cleanup(A)

    if label in visited and visited[label][0] <= distance:
        return visited[label][1]

    visited[label] = [distance, 99]

    neighbours = list(filter(lambda x: checkEdge(G, A, x, threshold), xs))

    if len(neighbours) == 0:
        return 99

    dis = min(
        list(
            map(
                lambda a: shortestPath(G, a, B, xs, distance + 1, threshold, memory),
                neighbours,
            )
        )
    )
    visited[label][1] = dis + 1
    return dis + 1

---


# Tests


In [12]:
def testCheckEdge(G, graph, threshold):
    count = 0

    for n in graph.nodes():
        for m in graph.nodes():
            if n == m:
                continue
            exist = graph.has_edge(n, m)
            check = checkEdge(
                G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], threshold
            )
            if exist != check:
                count += 1
                print(n, m, exist, check)

    print(count, "%.5f" % round(count / EDGES, 5))

In [13]:
def testNodeMemoryReconstruction(G, graph, iter=15):
    for i in range(iter):
        memsi = nodeMemoryReconstruction(
            G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
        )
        print(
            f"{0}_{i:02} =>",
            "%.10f" % abs(sim(graph.nodes[0]["mem"], memsi[0])),
        )

---


# Main


In [14]:
NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 10000, 0.047, 4

In [15]:
def main():
    graph = nx.gnm_random_graph(NODES, EDGES)
    memory = ItemMemory()

    initVertices(graph, DIMENSIONS)
    initNodeMem(graph, memory)
    retrain(graph, memory, THRESHOLD, ITER)
    G = initGraph(graph, memory)
    print("G =>", G)
    # testNodeMemoryReconstruction(G, graph, 1)
    # testCheckEdge(G, graph, THRESHOLD)


# main()

In [16]:
# graph = nx.gnm_random_graph(NODES, EDGES)
# memory = ItemMemory()

# initVertices(graph, DIMENSIONS)
# initNodeMem(graph, memory)
# G = initGraph(graph, memory)
# print("G =>", G)

G => [ 4. 10.  0. ... -4. 12. -4.]


In [17]:
# testNodeMemoryReconstruction(G, graph, 15)

0_00 => 0.2406504797
0_01 => 0.2406504797
0_02 => 0.2406504797
0_03 => 0.2406504797
0_04 => 0.2406504797
0_05 => 0.2406504797
0_06 => 0.2406504797
0_07 => 0.2406504797
0_08 => 0.2406504797
0_09 => 0.2406504797
0_10 => 0.2406504797
0_11 => 0.2406504797
0_12 => 0.2406504797
0_13 => 0.2406504797
0_14 => 0.2406504797


In [18]:
# sim(graph.nodes[0]["mem"], graph.nodes[0]["mem"])

0.9999999999999999

---


In [117]:
mem = ItemMemory()

g = nx.gnm_random_graph(NODES, 45)
initVertices(g, DIMENSIONS)
initNodeMem(g, mem)
G = initGraph(g, mem)

for n in g.nodes():
    print(g.edges(n))

[(0, 8), (0, 14), (0, 25), (0, 7)]
[(1, 4), (1, 19), (1, 14), (1, 7)]
[(2, 3), (2, 19), (2, 13), (2, 27), (2, 8)]
[(3, 2), (3, 23)]
[(4, 1), (4, 29)]
[(5, 7), (5, 12), (5, 24)]
[(6, 24), (6, 14), (6, 16), (6, 27)]
[(7, 5), (7, 17), (7, 1), (7, 0)]
[(8, 0), (8, 20), (8, 21), (8, 12), (8, 2)]
[(9, 29)]
[(10, 13), (10, 11), (10, 25)]
[(11, 10)]
[(12, 8), (12, 5), (12, 19)]
[(13, 10), (13, 2)]
[(14, 27), (14, 17), (14, 0), (14, 6), (14, 1)]
[(15, 29), (15, 20)]
[(16, 29), (16, 25), (16, 18), (16, 6)]
[(17, 23), (17, 14), (17, 7), (17, 28)]
[(18, 16), (18, 26)]
[(19, 2), (19, 1), (19, 12)]
[(20, 8), (20, 15)]
[(21, 8)]
[(22, 25)]
[(23, 17), (23, 24), (23, 3)]
[(24, 6), (24, 25), (24, 23), (24, 5)]
[(25, 22), (25, 16), (25, 24), (25, 10), (25, 26), (25, 0)]
[(26, 25), (26, 18)]
[(27, 14), (27, 6), (27, 2)]
[(28, 17)]
[(29, 16), (29, 9), (29, 15), (29, 4)]


In [119]:
hdvs = list(map(lambda x: g.nodes[x]["hdv"], g.nodes()))
origin = 9

for m in g.nodes():
    if origin == m:
        continue
    path = nx.shortest_path(g, origin, m)
    oracle = len(path) - 1
    visited = {}
    test = shortestPath(G, hdvs[origin], hdvs[m], hdvs, 0, THRESHOLD, mem)
    print(f"{origin}_{m:02} =>", oracle, test, "T  " if oracle == test else "F X", path)

9_00 => 4 4 T   [9, 29, 16, 25, 0]
9_01 => 3 7 F X [9, 29, 4, 1]
9_02 => 5 5 T   [9, 29, 16, 6, 27, 2]
9_03 => 6 6 T   [9, 29, 16, 25, 24, 23, 3]
9_04 => 2 2 T   [9, 29, 4]
9_05 => 5 5 T   [9, 29, 4, 1, 7, 5]
9_06 => 3 3 T   [9, 29, 16, 6]
9_07 => 4 5 F X [9, 29, 4, 1, 7]
9_08 => 4 4 T   [9, 29, 15, 20, 8]
9_10 => 4 4 T   [9, 29, 16, 25, 10]
9_11 => 5 5 T   [9, 29, 16, 25, 10, 11]
9_12 => 5 5 T   [9, 29, 15, 20, 8, 12]
9_13 => 5 5 T   [9, 29, 16, 25, 10, 13]
9_14 => 4 4 T   [9, 29, 16, 6, 14]
9_15 => 2 2 T   [9, 29, 15]
9_16 => 2 2 T   [9, 29, 16]
9_17 => 5 7 F X [9, 29, 16, 6, 14, 17]
9_18 => 3 3 T   [9, 29, 16, 18]
9_19 => 4 6 F X [9, 29, 4, 1, 19]
9_20 => 3 3 T   [9, 29, 15, 20]
9_21 => 5 5 T   [9, 29, 15, 20, 8, 21]
9_22 => 4 4 T   [9, 29, 16, 25, 22]
9_23 => 5 5 T   [9, 29, 16, 6, 24, 23]
9_24 => 4 4 T   [9, 29, 16, 25, 24]
9_25 => 3 3 T   [9, 29, 16, 25]
9_26 => 4 4 T   [9, 29, 16, 25, 26]
9_27 => 4 4 T   [9, 29, 16, 6, 27]
9_28 => 6 8 F X [9, 29, 16, 6, 14, 17, 28]
9_29 => 1 1 T